<a href="https://colab.research.google.com/github/EmilyCarroll-del/Michael-J-Fox-Foundation-FOG-in-PD/blob/main/MJF_LSTM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
%cd '/content/drive/MyDrive/Colab Notebooks/'
%ls

/content/drive/MyDrive/Colab Notebooks
defog_features.csv            tlvmc-parkinsons-freezing-gait-prediction/
MJFF-FOG-Prediction-PD.ipynb  tlvmc-parkinsons-freezing-gait-prediction.zip


In [25]:
pip install tsflex seglearn tensorflow

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from scipy.stats import zscore
from tsflex.features import FeatureCollection, MultipleFeatureDescriptors
from tsflex.features.integrations import seglearn_feature_dict_wrapper
from seglearn.feature_functions import base_features, emg_features
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [5]:
def process_directory(directory):
    dfs = []
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)

        # Check if the file is a CSV and process it
        if file_name.endswith('.csv') and os.path.isfile(file_path):
            df = pd.read_csv(file_path) #read csv files
            df['source_directory'] = os.path.basename(directory)  # Add a column to identify the source directory
            df['csv_name'] = os.path.basename(file_name)
            dfs.append(df)

        return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

In [6]:
defog = process_directory('/content/drive/MyDrive/Colab Notebooks/tlvmc-parkinsons-freezing-gait-prediction/train/defog')
tdcsfog = process_directory('/content/drive/MyDrive/Colab Notebooks/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog')
notype = process_directory('/content/drive/MyDrive/Colab Notebooks/tlvmc-parkinsons-freezing-gait-prediction/train/notype')

In [21]:
#create IsFOG column
defog['IsFOG'] = defog[['StartHesitation', 'Walking', 'Turn']].any(axis='columns')
defog['IsFOG'] = defog['IsFOG'].astype(int)
tdcsfog['IsFOG'] = tdcsfog[['StartHesitation', 'Walking', 'Turn']].any(axis='columns')
tdcsfog['IsFOG'] = tdcsfog['IsFOG'].astype(int)

#removing missing values
#defog dataset, valid = false, task = false
#tdcsfog has no missing values
defog = defog[defog['Valid']]
defog = defog[defog['Task']]

In [18]:
def analyze_events_by_subject(df, dataset_name):
    # Group by 'csv_name' assuming each file corresponds to a subject
    events_by_subject = df.groupby('csv_name')[['Turn', 'Walking', 'StartHesitation']].sum()

    # Calculate the total number of timestamps for each subject
    total_timestamps = df.groupby('csv_name').size()

    # Add the total timestamps to the dataframe
    events_by_subject['Total'] = total_timestamps

    # Calculate percentages
    for event in ['Turn', 'Walking', 'StartHesitation']:
        events_by_subject[f'{event}_Percent'] = events_by_subject[event] / events_by_subject['Total'] * 100

    # Sort by total timestamps
    events_by_subject = events_by_subject.sort_values('Total', ascending=False)

    return events_by_subject

defog_events = analyze_events_by_subject(defog, 'DEFOG')
tdcsfog_events = analyze_events_by_subject(tdcsfog, 'TDCSFOG')

                Turn  Walking  StartHesitation  Total  Turn_Percent  \
csv_name                                                              
02ea782681.csv   129        0                0  36873      0.349849   

                Walking_Percent  StartHesitation_Percent  
csv_name                                                  
02ea782681.csv              0.0                      0.0  


In [8]:
# Outlier detection function using IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

outliers_AccV_defog = detect_outliers_iqr(defog, 'AccV')
outliers_AccML_defog = detect_outliers_iqr(defog, 'AccML')
outliers_AccAP_defog = detect_outliers_iqr(defog, 'AccAP')

outliers_AccV_tdcsfog = detect_outliers_iqr(tdcsfog, 'AccV')
outliers_AccML_tdcsfog = detect_outliers_iqr(tdcsfog, 'AccML')
outliers_AccAP_tdcsfog = detect_outliers_iqr(tdcsfog, 'AccAP')

In [9]:
#DISCUSS THIS CODE BLOCK, output = empty df
#defog['AccAP_zscore'] = zscore(defog['AccAP'])
#defog['Walking_zscore'] = zscore(defog['Walking'])
#print(defog[['AccAP', 'AccAP_zscore', 'Walking', 'Walking_zscore']].head())

#threshold = 0.1
#filtered_defog = defog[(defog['AccAP_zscore'].abs() <= threshold) & (defog['Walking_zscore'].abs() <= threshold)]
#print(filtered_defog[['AccAP', 'AccAP_zscore', 'Walking', 'Walking_zscore']].head())

In [13]:
#extract TIME and FREQUENCY-DOMAIN FEATURES for DEFOG
basic_feats = MultipleFeatureDescriptors(
    functions=seglearn_feature_dict_wrapper(base_features()),
    series_names=['AccV', 'AccML', 'AccAP'],
    windows=[500],
    strides=[500],
)

emg_feats = emg_features()
del emg_feats['simple square integral'] # is same as abs_energy (which is in base_features)

emg_feats = MultipleFeatureDescriptors(
    functions=seglearn_feature_dict_wrapper(emg_feats),
    series_names=['AccV', 'AccML', 'AccAP'],
    windows=[500],
    strides=[500]
)

fc = FeatureCollection([basic_feats, emg_feats])

#defog_feats = fc.calculate(defog, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin").astype(np.float32)

In [11]:
#defog = pd.concat([defog, defog_feats], axis=1)
#defog.head()

In [14]:
#extract TIME and FREQUENCY-DOMAIN FEATURES for DEFOG
basic_feats = MultipleFeatureDescriptors(
    functions=seglearn_feature_dict_wrapper(base_features()),
    series_names=['AccV', 'AccML', 'AccAP'],
    windows=[500],
    strides=[500],
)

emg_feats = emg_features()
del emg_feats['simple square integral'] # is same as abs_energy (which is in base_features)

emg_feats = MultipleFeatureDescriptors(
    functions=seglearn_feature_dict_wrapper(emg_feats),
    series_names=['AccV', 'AccML', 'AccAP'],
    windows=[500],
    strides=[500]
)

fc = FeatureCollection([basic_feats, emg_feats])

#tdcsfog_feats = fc.calculate(tdcsfog, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin").astype(np.float32)

In [22]:
#tdcsfog = pd.concat([tdcsfog, tdcsfog_feats], axis=1)
#tdcsfog.head()
defog.head()

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Valid,Task,source_directory,csv_name,IsFOG
1000,1000,-0.970018,0.061626,-0.265625,0,0,0,True,True,defog,02ea782681.csv,0
1001,1001,-0.984375,0.044497,-0.265625,0,0,0,True,True,defog,02ea782681.csv,0
1002,1002,-0.984375,0.029016,-0.265625,0,0,0,True,True,defog,02ea782681.csv,0
1003,1003,-0.984375,0.015625,-0.265625,0,0,0,True,True,defog,02ea782681.csv,0
1004,1004,-0.984670,0.015330,-0.265625,0,0,0,True,True,defog,02ea782681.csv,0


In [38]:
#training an LSTM model on defog

X = defog[['AccV', 'AccML', 'AccAP']].values  # Using accelerometer features
y = defog['IsFOG'].values

#samples: Number of windows/segments (i.e., the number of time series segments).
#time_steps: Number of time steps in each window/segment.
#n_features: Number of features (e.g., accelerometer axes: AccV, AccML, AccAP).
time_steps = 1  #needs to be adjusted
n_features = X.shape[1]

num_samples = X.shape[0]

# Calculate the number of complete windows
if num_samples % time_steps != 0:
    # If the data doesn't fit evenly into windows, trim the excess samples
    num_samples = (num_samples // time_steps) * time_steps
    X_trimmed = X[:num_samples]  # Trim X to be divisible by time_steps
    y_trimmed = y[:num_samples]
else:
    X_trimmed = X
    y_trimmed = y

X_reshaped = X_trimmed.reshape(-1, time_steps, n_features)  # Reshape to (samples, time steps, features)
y_reshaped = y_trimmed[:num_samples].reshape(-1, time_steps)

X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_reshaped, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = np.array([scaler.fit_transform(seq) for seq in X_train])  # Normalize each sequence
X_test = np.array([scaler.transform(seq) for seq in X_test])

In [39]:
model = Sequential()

# Add LSTM layer
model.add(LSTM(units=64, input_shape=(time_steps, n_features), return_sequences=False))

# Add Dropout for regularization
model.add(Dropout(0.5))

# Add Dense layer (output layer)
model.add(Dense(1, activation='sigmoid'))  # For binary classification (use 'softmax' for multi-class)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary to see the architecture
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 64)                  │          17,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 17,473 (68.25 KB)

 Trainable params: 17,473 (68.25 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,  # Number of epochs to train
    batch_size=64,  # Batch size
    validation_data=(X_test, y_test),
    verbose=2  # Progress bar and logs
)

Epoch 1/20
461/461 - 4s - 9ms/step - accuracy: 0.9963 - loss: 0.2217 - val_accuracy: 0.9972 - val_loss: 0.0294
Epoch 2/20
461/461 - 2s - 5ms/step - accuracy: 0.9963 - loss: 0.0278 - val_accuracy: 0.9972 - val_loss: 0.0202
Epoch 3/20
461/461 - 2s - 5ms/step - accuracy: 0.9963 - loss: 0.0249 - val_accuracy: 0.9972 - val_loss: 0.0196
Epoch 4/20
461/461 - 2s - 4ms/step - accuracy: 0.9963 - loss: 0.0252 - val_accuracy: 0.9972 - val_loss: 0.0196
Epoch 5/20
461/461 - 2s - 5ms/step - accuracy: 0.9963 - loss: 0.0250 - val_accuracy: 0.9972 - val_loss: 0.0196
Epoch 6/20
461/461 - 2s - 5ms/step - accuracy: 0.9963 - loss: 0.0251 - val_accuracy: 0.9972 - val_loss: 0.0196
Epoch 7/20
461/461 - 2s - 4ms/step - accuracy: 0.9963 - loss: 0.0250 - val_accuracy: 0.9972 - val_loss: 0.0196
Epoch 8/20
461/461 - 1s - 3ms/step - accuracy: 0.9963 - loss: 0.0252 - val_accuracy: 0.9972 - val_loss: 0.0196
Epoch 9/20
461/461 - 2s - 5ms/step - accuracy: 0.9963 - loss: 0.0251 - val_accuracy: 0.9972 - val_loss: 0.0196
E

In [41]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

231/231 - 0s - 2ms/step - accuracy: 0.9972 - loss: 0.0196
Test Loss: 0.0196, Test Accuracy: 0.9972
